In [1]:
print('Validate nvidia-smi (software managemtn interface):')
!nvidia-smi
print()
print('Validate nvcc --version:')
!nvcc --version

Validate nvidia-smi (software managemtn interface):
Wed Oct 18 11:31:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P8     6W /  N/A |      8MiB /  3910MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                    

In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras.models import Sequential
# Both tensorflow LSTM and NVIDIA cudaDNN have the GPU LSTM cell.
# NIVIDIA cudaDNN even have more optimized LSTM cell with 5 x faster. 
from tensorflow.keras.layers import Dense, Dropout, LSTM

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train.shape:', x_train.shape)
print('x_train[0].shape:', x_train[0].shape)
print('x_train[0][5:10]:')
print(x_train[0][5:10])

x_train.shape: (60000, 28, 28)
x_train[0].shape: (28, 28)
x_train[0][5:10]:
[[  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  80 156 107 253 253 205  11   0  43 154
    0   0   0   0   0   0   0   0   0   0]]


In [26]:
x_train = x_train/250.0
x_test = x_test/255.0

model = Sequential ()
# we add the 128 LSTM cell, 
print('x_train.shape[1:]:', x_train.shape[1:])
# x_train.shape[1:] = (28, 28)
# We can do cudaNNLSTM later on.
# model.add(cudaNNLSTM(128, input_shape=(x_train.shape[1:]), activation='relu'))
model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences = True))
model.add(Dropout(0.2)) # we dropout 20% neurons.
# reurn_sequences = True is only use to the output layer.
# If go the another layer, we don't need return_sequences

model.add(LSTM(128, activation='relu')) # hidden layer does not need the input_shape
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))  # Make 32 nodes in the dense layer.
model.add(Dropout(0.2))

# Make 10 nodes in the output layer and softmax in the output proability distribution
model.add(Dense(10, activation='softmax'))
# set up parameters, learning rate small step (1e-3) is better for convergence.
# learning rate keep on shrinking (decay) 1e-5
# opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
# decay_rate = learning_rate / epochs
# https://stackoverflow.com/questions/74734685/how-to-fix-this-value-error-valueerror-decay-is-deprecated-in-the-new-keras-o
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100000,
    decay_rate=1e-5)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])
model.fit (x_train, y_train, epochs=3, validation_data = (x_test, y_test))

x_train.shape[1:]: (28, 28)
Epoch 1/3


2023-10-19 15:07:14.148027: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 48s 25ms/step - loss: 0.6317 - accuracy: 0.7924 - val_loss: 0.1565 - val_accuracy: 0.9526
Epoch 2/3
1875/1875 [==============================] - 48s 25ms/step - loss: 0.1631 - accuracy: 0.9565 - val_loss: 0.1023 - val_accuracy: 0.9696
Epoch 3/3
1875/1875 [==============================] - 47s 25ms/step - loss: 0.1359 - accuracy: 0.9652 - val_loss: 0.0981 - val_accuracy: 0.9755


In [31]:
# We change LSTM into NVIDIA cuDNNLSTM
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras.models import Sequential
# Both tensorflow LSTM and NVIDIA cudaDNN have the GPU LSTM cell.
# NIVIDIA cudaDNN even have more optimized LSTM cell with 5 x faster. 
# We don't need to import CuDNNLSTM since Tensorflow 2 and NVIDIA CuDNNLSTM already merge together.
# from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM
# ImportError: cannot import name 'CuDNNLSTM' from 'tensorflow.keras.layers'
# https://forums.developer.nvidia.com/t/importerror-cannot-import-name-cudnnlstm-from-tensorflow-keras-layers/82778/2
# The error was because from TensorFlow 2 you do not need to specify CuDNNLSTM. 
# You can just use LSTM with no activation function and it will automatically 
# use the CuDNN version. 
# You do have to install CuDNN first.
from tensorflow.keras.layers import Dense, Dropout, LSTM
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train.shape:', x_train.shape)
print('x_train[0].shape:', x_train[0].shape)
print('x_train[0][5:10]:')
print(x_train[0][5:10])

x_train = x_train/250.0
x_test = x_test/255.0

model = Sequential ()
print('x_train.shape[1:]:', x_train.shape[1:])
model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences = True))
model.add(Dropout(0.2)) # we dropout 20% neurons.

# CuDNN does not need to specify the activation function.
model.add(LSTM(128)) # hidden layer does not need the input_shape
model.add(Dropout(0.2))

model.add(Dense(32))  # Make 32 nodes in the dense layer.
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100000,
    decay_rate=1e-5)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])
model.fit (x_train, y_train, epochs=3, validation_data = (x_test, y_test))

x_train.shape: (60000, 28, 28)
x_train[0].shape: (28, 28)
x_train[0][5:10]:
[[  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  80 156 107 253 253 205  11   0  43 154
    0   0   0   0   0   0   0   0   0   0]]
x_train.shape[1:]: (28, 28)
Epoch 1/3


2023-10-19 15:29:12.669629: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 48s 25ms/step - loss: 0.3249 - accuracy: 0.8970 - val_loss: 0.0834 - val_accuracy: 0.9739
Epoch 2/3
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1013 - accuracy: 0.9710 - val_loss: 0.0706 - val_accuracy: 0.9787
Epoch 3/3
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0686 - accuracy: 0.9805 - val_loss: 0.0509 - val_accuracy: 0.9822
